## Customer Churn Prediction
### Lifecycle of customer relationship
A typical customer relationship goes through a number of phases.  

![](images/churn.png)
 
- In the endangerment phase the risk for customer churn increases.
- Companies can avoid churn by predicting it’s risk for each individual customer.
- Customers that are at risk can get special attention: discounts, extra service, etc. 
  
### Predictive models for customer churn
- Customer churn can be predicted using machine learning techniques.
- Predicting customer churn is a typical classification problem.
- Often Decision Trees or Naïve Bayes are used.

### Case: customer churn at a telecom provider
- VODAK is a supplier of a mobile services in the Netherlands. 
- The marketingmanager would like to get more insights in customer loyalty and the factors determining switch behaviour 
- VODAK has a database of over one million customers.
- We have a sample of these database with 7001 customers. 
- Based on this sample we will provide the marketing manager with the necessary insights and predict which customers might leave the company.

### Case telecom: data description
- N° of months since customer started
- Age
- Gender (M/F)
- Average monthly income in euros per customer
- N° of months since start of current contract
- N° of months till end of contract
- N° of times customer switched phone
- Age of head of household
- Education level of head of household
- Income of head of household
- House type
- Working situation
- N° of calls last month
- Used minutes last month
- Sent text messages last month
- Average n° of calls last 6 months
- Average used minutes last 6 months
- Average n° of sent text messages last 6 months
- Service contract
- Acquisition costs
- Switch: has customer cancelled contract? 

### Steps to build, interpret and use model
- Determine and drop features that seem irrelevant (based on common sense)
- Handle missing data
- Use one-hot-encoding for categorical features
- Choose prediction algorithm
- Determine the label
- Build model
- Determine accuracy of model
- Determine relative importance of features and interpret results
- Write a function to determine  the probability that a (unseen) customer will leave.




In [2]:
# import the datafile 
import pandas as pd
url = 'https://raw.githubusercontent.com/HOGENT-Databases/DB3-Workshops/master/data/telecom_case.csv'
telecom = pd.read_csv(url,sep=',')

In [3]:
print(telecom.head(20))

    no_months   age gender  income  no_months_ctr  no_months_ctr_end  phones  \
0          73   NaN    NaN   47.00           30.0               -6.0       4   
1          33   NaN    NaN   21.01           21.0               -9.0       3   
2         101   NaN    NaN  132.49           22.0               11.0       3   
3         132   NaN    NaN   32.09           26.0               -9.0       6   
4         112  47.0    NaN    3.63           34.0              -10.0       1   
5         112  48.0      M    0.48           49.0              -25.0       2   
6          56   NaN    NaN   21.61           32.0               -8.0       1   
7         109   NaN    NaN   84.07           13.0               10.0       3   
8          85   NaN    NaN   64.48           39.0               -9.0       5   
9         108  69.0      M    0.00           28.0              -15.0       2   
10        115  52.0      M   16.61           32.0               -8.0       2   
11        114  53.0    NaN  129.07      

In [4]:
telecom.groupby(telecom.switch).count()

,no_months,age,gender,income,no_months_ctr,no_months_ctr_end,phones,age_head_hh,edu_head_hh,income_head_hh,house_type,work_situation,no_calls,no_minutes,no_msgs,no_calls6,no_minutes6,no_msgs6,servcon,acq_costs
switch,,,,,,,,,,,,,,,,,,,,
0,4788,4301,4282,4788,4781,4781,4788,4253,4253,4253,4253,4253,4788,4788,4788,4788,4788,4788,4788,4788
1,2213,2005,1980,2213,2208,2208,2213,2052,2052,2052,2052,2052,2213,2213,2213,2213,2213,2213,2213,2213


In [5]:
# drop features that seem irrelevant
telecom = telecom.drop(['servcon','acq_costs'],axis=1)

In [6]:
# Remove features with many missing values
# Identify missing values
telecom.isna()


,no_months,age,gender,income,no_months_ctr,no_months_ctr_end,phones,age_head_hh,edu_head_hh,income_head_hh,house_type,work_situation,no_calls,no_minutes,no_msgs,no_calls6,no_minutes6,no_msgs6,switch
0,False,True,True,False,False,False,False,True,True,True,True,True,False,False,False,False,False,False,False
1,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6996,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6997,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6998,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6999,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [7]:
# count missing values
telecom.isna().sum()

no_months              0
age                  695
gender               739
income                 0
no_months_ctr         12
no_months_ctr_end     12
phones                 0
age_head_hh          696
edu_head_hh          696
income_head_hh       696
house_type           696
work_situation       696
no_calls               0
no_minutes             0
no_msgs                0
no_calls6              0
no_minutes6            0
no_msgs6               0
switch                 0
dtype: int64

In [8]:
# Count missing value as proportion of total number of rows
telecom.isna().sum()/len(telecom)

no_months            0.000000
age                  0.099272
gender               0.105556
income               0.000000
no_months_ctr        0.001714
no_months_ctr_end    0.001714
phones               0.000000
age_head_hh          0.099414
edu_head_hh          0.099414
income_head_hh       0.099414
house_type           0.099414
work_situation       0.099414
no_calls             0.000000
no_minutes           0.000000
no_msgs              0.000000
no_calls6            0.000000
no_minutes6          0.000000
no_msgs6             0.000000
switch               0.000000
dtype: float64

In [9]:
# Apply a missing value threshold
mask = telecom.isna().sum()/len(telecom) < 0.1
print(mask)

no_months             True
age                   True
gender               False
income                True
no_months_ctr         True
no_months_ctr_end     True
phones                True
age_head_hh           True
edu_head_hh           True
income_head_hh        True
house_type            True
work_situation        True
no_calls              True
no_minutes            True
no_msgs               True
no_calls6             True
no_minutes6           True
no_msgs6              True
switch                True
dtype: bool


In [10]:
# Remove columns with a missing value proportion above this threshold
telecom = telecom.loc[:,mask]
telecom.head()

,no_months,age,income,no_months_ctr,no_months_ctr_end,phones,age_head_hh,edu_head_hh,income_head_hh,house_type,work_situation,no_calls,no_minutes,no_msgs,no_calls6,no_minutes6,no_msgs6,switch
0,73,NaN,47.00,30.0,-6.0,4,NaN,NaN,NaN,NaN,NaN,0,0.00,0,253,352.78,211,1
1,33,NaN,21.01,21.0,-9.0,3,5.0,1.0,5.0,1.0,5.0,1,0.43,0,67,59.95,6,1
2,101,NaN,132.49,22.0,11.0,3,4.0,3.0,5.0,1.0,1.0,0,0.00,0,78,86.27,4,1
3,132,NaN,32.09,26.0,-9.0,6,4.0,2.0,2.0,4.0,1.0,0,0.00,0,180,314.88,16,1
4,112,47.0,3.63,34.0,-10.0,1,4.0,1.0,3.0,2.0,1.0,16,14.50,0,425,452.47,10,1


In [11]:
# drop lines with unknown (NaN) values
telecom = telecom.dropna()

In [12]:
telecom.shape

(5692, 18)

In [13]:
# determine feature set and label

X = telecom.drop('switch',axis=1)
y = telecom['switch']

In [14]:
# build model (see course Databases III)
from sklearn.model_selection import train_test_split
X_remainder, X_test, y_remainder, y_test = train_test_split(X,y,test_size=0.30)

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

best_accuracy = 0
best_trees = 0

for trees in range(50,550,50):
    X_train, X_validation, y_train, y_validation = train_test_split(X_remainder,y_remainder,test_size=0.30)
    model = RandomForestClassifier(n_estimators=trees)
    model.fit(X_train, y_train)    
    y_validation2 = model.predict(X_validation)
    accuracy = accuracy_score(y_validation, y_validation2)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_trees = trees
        best_validation = model.predict(X_test)
        
print('Optimal number of trees = % s' %(best_trees))
print('Accuracy on validation set = % 3.2f' % (best_accuracy)) 
accuracyOnTestSet = accuracy_score(y_test, best_validation)
print('Accuracy on test set = % 3.2f' % (accuracyOnTestSet))

Optimal number of trees = 50
Accuracy on validation set =  0.99
Accuracy on test set =  0.99


In [15]:
# determine feature importances
print(X_train.columns)
print(model.feature_importances_)

Index(['no_months', 'age', 'income', 'no_months_ctr', 'no_months_ctr_end',
       'phones', 'age_head_hh', 'edu_head_hh', 'income_head_hh', 'house_type',
       'work_situation', 'no_calls', 'no_minutes', 'no_msgs', 'no_calls6',
       'no_minutes6', 'no_msgs6'],
      dtype='object')
[0.00584028 0.00630422 0.06370116 0.01688216 0.03475929 0.00256941
 0.00179856 0.00127057 0.00410253 0.00157651 0.00086732 0.32910003
 0.33644661 0.16691265 0.01188065 0.01069577 0.0052923 ]


In [16]:
pd.DataFrame(model.feature_importances_,columns=['Importance'],index=X_train.columns).sort_values(by='Importance',ascending=False)

,Importance
no_minutes,0.336447
no_calls,0.329100
no_msgs,0.166913
income,0.063701
no_months_ctr_end,0.034759
no_months_ctr,0.016882
no_calls6,0.011881
no_minutes6,0.010696
age,0.006304
no_months,0.005840


In [19]:
# are persons with a higher or lower no_minutes, etc. more likely to switch? 
telecom.groupby(telecom.switch).mean()

,no_months,age,income,no_months_ctr,no_months_ctr_end,phones,age_head_hh,edu_head_hh,income_head_hh,house_type,work_situation,no_calls,no_minutes,no_msgs,no_calls6,no_minutes6,no_msgs6
switch,,,,,,,,,,,,,,,,,
0,52.545668,39.345511,38.622035,21.602296,1.553497,2.269833,3.371347,2.093946,3.215553,3.191806,1.472338,85.149530,125.893392,60.675365,93.192850,138.123700,64.942589
1,49.644086,36.088710,201.819565,26.468280,-4.209140,2.591935,3.312903,2.072581,2.629032,3.531720,1.540860,0.159677,0.210167,0.091398,120.936559,180.063839,52.639247


In [18]:
# We will now use this model to predict wether or not some current customers might leave or not.
# This will typically be part of a end-user application and run e.g. weekly on the customer database

def PredictSwitch(model,no_months,age,income,no_months_ctr,no_months_ctr_end,phones,age_head_hh,  
                  edu_head_hh, income_head_hh,house_type,work_situation,no_calls,no_minutes,
                  no_msgs,no_calls6,no_minutes6,no_msgs6):
    import pandas as pd
    customer=pd.DataFrame(columns=['no_months','age','income','no_months_ctr','no_months_ctr_end',
    'phones','age_head_hh','edu_head_hh','income_head_hh','house_type','work_situation',
    'no_calls','no_minutes','no_msgs','no_calls6','no_minutes6','no_msgs6'])

    new_customer = {'no_months':no_months,
                    'age':age,
                    'income':income,
                    'no_months_ctr':no_months_ctr,
                    'no_months_ctr_end':no_months_ctr_end,
                    'phones':phones,
                    'age_head_hh':age_head_hh,
                    'edu_head_hh':edu_head_hh,
                    'income_head_hh':income_head_hh,
                    'house_type':house_type,
                    'work_situation':work_situation,
                    'no_calls':no_calls,
                    'no_minutes':no_minutes,
                    'no_msgs':no_msgs,
                    'no_calls6':no_calls6,
                    'no_minutes6':no_minutes6,
                    'no_msgs6':no_msgs6}
    
    customer = customer.append(new_customer,ignore_index=True)
    # In practice the model will be saved to a file after building and fine-tuning 
    # and loaded from that file in this function
    switch = model.predict(customer)
    
    # most sklearn algorithms also offer a predict_proba method that returns an array of 
    # probabilities per class:
    switch_proba = model.predict_proba(customer)
    return switch[0],switch_proba[0].max()


switch = PredictSwitch(model,no_months=28,age=21,income=531.35,no_months_ctr=17,no_months_ctr_end=15,
                       phones=1,age_head_hh=3,edu_head_hh=2,income_head_hh=1,house_type=5,
                       work_situation=1,no_calls=0,no_minutes=0,
                       no_msgs=0,no_calls6=106,no_minutes6=98.02,no_msgs6=15)

print(switch)

switch = PredictSwitch(model,no_months=24,age=46,income=300,no_months_ctr=20,no_months_ctr_end=7,
        phones=2,age_head_hh=46,edu_head_hh=2,income_head_hh=4,house_type=5,work_situation=1,       
        no_calls=50,no_minutes=400,no_msgs=5,no_calls6=70,no_minutes6=8,no_msgs6=10)

print(switch)



(1, 0.944)
(0, 0.98)
